MAE - Mean Absolute Error (lower is better)

RMSE - Root Mean Square Error (if RMSE >> MAE there is large outliers)

Bias - bias > 0 (model systematically overpredicts); bias < 0 (model underpredicts); bias = 0 (unbiased)

Correlation - corr = 1 (perfect temporal evolution); corr = 0 (no skill); corr < 0 (wrong dynamics)

In [1]:
# ============================================================
# DAILY POLLUTANT PREDICTION WITH LSTM (SINGLE POLLUTANT)
# NaNs are strictly ignored (no interpolation, no filling)
# ============================================================

import numpy as np
import xarray as xr

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# FILE PATH (CHANGE THIS PER RUN)
# ============================================================

POLLUTANT_NAME = "CO"   # for naming outputs only
INPUT_FILE = r"D:\IPMA\Results\co_fire_meteo_Iberia.nc"

# ============================================================
# USER SETTINGS
# ============================================================

POLLUTANT_VAR = "Mean"
TEST_YEAR = 2017
TEST_MONTH = None        # None or 1–12
LAG_DAYS = 14
EPOCHS = 40
BATCH_SIZE = 32

INPUT_VARS = [
    "Mean",
    "temp_Max",
    "wind_Max",
    "precip_Total_Precipitation",
    "frp_sum_Iberia"
]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def build_sequences(X, y, lags):
    X_out, y_out = [], []
    for i in range(lags, len(X)):
        X_out.append(X[i-lags:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)


def train_mask(time):
    return time.dt.year != TEST_YEAR


def test_mask(time):
    mask = time.dt.year == TEST_YEAR
    if TEST_MONTH is not None:
        mask = mask & (time.dt.month == TEST_MONTH)
    return mask

# ============================================================
# LOAD DATA
# ============================================================

print(f"\n==============================")
print(f"Processing: {POLLUTANT_NAME}")
print(f"==============================")

ds = xr.open_dataset(INPUT_FILE)

# -------------------------------
# CHECK VARIABLES
# -------------------------------

for var in INPUT_VARS:
    if var not in ds:
        raise KeyError(f"{var} not found in {INPUT_FILE}")

y = ds[POLLUTANT_VAR]
X = xr.merge([ds[var] for var in INPUT_VARS])

time = ds.time
train_idx = train_mask(time)
test_idx = test_mask(time)

X_train = X.sel(time=train_idx)
y_train = y.sel(time=train_idx)
X_test = X.sel(time=test_idx)
y_test = y.sel(time=test_idx)

# ============================================================
# BUILD TRAINING DATA (NaN-SAFE)
# ============================================================

X_all, y_all = [], []

for lat in ds.latitude.values:
    for lon in ds.longitude.values:

        X_ts = X_train.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y_train.sel(latitude=lat, longitude=lon).values

        valid = (
            ~np.isnan(y_ts) &
            ~np.isnan(X_ts).any(axis=1)
        )

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        if len(y_ts) <= LAG_DAYS:
            continue

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).ravel()

        X_seq, y_seq = build_sequences(X_scaled, y_scaled, LAG_DAYS)

        if len(y_seq) == 0:
            continue

        X_all.append(X_seq)
        y_all.append(y_seq)

X_train_all = np.concatenate(X_all)
y_train_all = np.concatenate(y_all)

print(f"Training samples: {X_train_all.shape[0]}")

# ============================================================
# MODEL
# ============================================================

model = Sequential([
    LSTM(64, input_shape=(LAG_DAYS, X_train_all.shape[2])),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")

model.fit(
    X_train_all,
    y_train_all,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

# ============================================================
# PREDICTION (NaN-SAFE)
# ============================================================

test_times = time.sel(time=test_idx)

preds = np.full(
    (len(ds.latitude), len(ds.longitude), len(test_times)),
    np.nan
)

for i, lat in enumerate(ds.latitude.values):
    for j, lon in enumerate(ds.longitude.values):

        X_ts = X.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y.sel(latitude=lat, longitude=lon).values

        valid = ~np.isnan(X_ts).any(axis=1)

        if valid.sum() <= LAG_DAYS:
            continue

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1))

        test_start = np.where(test_idx.values)[0][0]

        for k in range(len(test_times)):
            t = test_start + k
            if t - LAG_DAYS < 0:
                continue

            X_input = X_scaled[t-LAG_DAYS:t]

            if np.isnan(X_input).any():
                continue

            pred_scaled = model.predict(
                X_input.reshape(1, LAG_DAYS, -1),
                verbose=0
            )

            preds[i, j, k] = scaler_y.inverse_transform(pred_scaled)[0, 0]

# ============================================================
# OUTPUT
# ============================================================

pred_da = xr.DataArray(
    preds,
    dims=("latitude", "longitude", "time"),
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times
    },
    name=f"{POLLUTANT_NAME}_predicted"
)

# ============================================================
# EVALUATION (NaN-SAFE)
# ============================================================

error = pred_da - y_test

mae = float(abs(error).where(np.isfinite(error)).mean())
rmse = float(np.sqrt((error ** 2)).where(np.isfinite(error)).mean())
bias = float(error.where(np.isfinite(error)).mean())
corr = float(
    xr.corr(pred_da, y_test, dim="time")
    .where(np.isfinite(pred_da))
    .mean()
)

print("\nEvaluation (2017):")
print(f"  MAE:  {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  Bias: {bias:.4f}")
print(f"  Corr: {corr:.4f}")

# ============================================================
# SAVE RESULTS FOR MAPPING (NO PLOTTING)
# ============================================================

ds_out = xr.Dataset(
    {
        f"{POLLUTANT_NAME}_predicted": pred_da,
        f"{POLLUTANT_NAME}_observed": y_test,
        f"{POLLUTANT_NAME}_difference": pred_da - y_test,
    },
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times,
    },
)

# Add metadata (recommended)
ds_out[f"{POLLUTANT_NAME}_predicted"].attrs["description"] = "LSTM predicted pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_observed"].attrs["description"] = "Observed pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_difference"].attrs["description"] = "Prediction minus observation"

ds_out.attrs["model"] = "LSTM"
ds_out.attrs["lags_days"] = LAG_DAYS
ds_out.attrs["test_year"] = TEST_YEAR
ds_out.attrs["pollutant"] = POLLUTANT_NAME

# Save to NetCDF
output_file = f"{POLLUTANT_NAME}_LSTM_predictions_{TEST_YEAR}.nc"
ds_out.to_netcdf(output_file)

print(f"\nSaved results to: {output_file}")



C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):



Processing: CO
Training samples: 1139999


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 162s 5ms/step - loss: 0.0012
Epoch 2/40
   22/35625 ━━━━━━━━━━━━━━━━━━━━ 3:03 5ms/step - loss: 8.2919e-04 

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


35625/35625 ━━━━━━━━━━━━━━━━━━━━ 181s 5ms/step - loss: 0.0011
Epoch 3/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 125s 4ms/step - loss: 0.0011
Epoch 4/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 126s 4ms/step - loss: 0.0011
Epoch 5/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 128s 4ms/step - loss: 0.0011
Epoch 6/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 127s 4ms/step - loss: 0.0011
Epoch 7/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 127s 4ms/step - loss: 0.0010
Epoch 8/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 127s 4ms/step - loss: 0.0010
Epoch 9/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 127s 4ms/step - loss: 0.0010
Epoch 10/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 128s 4ms/step - loss: 0.0010
Epoch 11/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 127s 4ms/step - loss: 0.0010
Epoch 12/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 128s 4ms/step - loss: 0.0010
Epoch 13/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 127s 4ms/step - loss: 9.9870e-04
Epoch 14/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/step - loss: 9.9069e-04
Epoch 15/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 120s

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [2]:
# ============================================================
# DAILY POLLUTANT PREDICTION WITH LSTM (SINGLE POLLUTANT)
# NaNs are strictly ignored (no interpolation, no filling)
# ============================================================

import numpy as np
import xarray as xr

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# FILE PATH (CHANGE THIS PER RUN)
# ============================================================

POLLUTANT_NAME = "NO"   # for naming outputs only
INPUT_FILE = r"D:\IPMA\Results\no_fire_meteo_Iberia.nc"

# ============================================================
# USER SETTINGS
# ============================================================

POLLUTANT_VAR = "Mean"
TEST_YEAR = 2017
TEST_MONTH = None        # None or 1–12
LAG_DAYS = 14
EPOCHS = 40
BATCH_SIZE = 32

INPUT_VARS = [
    "Mean",
    "temp_Max",
    "wind_Max",
    "precip_Total_Precipitation",
    "frp_sum_Iberia"
]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def build_sequences(X, y, lags):
    X_out, y_out = [], []
    for i in range(lags, len(X)):
        X_out.append(X[i-lags:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)


def train_mask(time):
    return time.dt.year != TEST_YEAR


def test_mask(time):
    mask = time.dt.year == TEST_YEAR
    if TEST_MONTH is not None:
        mask = mask & (time.dt.month == TEST_MONTH)
    return mask

# ============================================================
# LOAD DATA
# ============================================================

print(f"\n==============================")
print(f"Processing: {POLLUTANT_NAME}")
print(f"==============================")

ds = xr.open_dataset(INPUT_FILE)

# -------------------------------
# CHECK VARIABLES
# -------------------------------

for var in INPUT_VARS:
    if var not in ds:
        raise KeyError(f"{var} not found in {INPUT_FILE}")

y = ds[POLLUTANT_VAR]
X = xr.merge([ds[var] for var in INPUT_VARS])

time = ds.time
train_idx = train_mask(time)
test_idx = test_mask(time)

X_train = X.sel(time=train_idx)
y_train = y.sel(time=train_idx)
X_test = X.sel(time=test_idx)
y_test = y.sel(time=test_idx)

# ============================================================
# BUILD TRAINING DATA (NaN-SAFE)
# ============================================================

X_all, y_all = [], []

for lat in ds.latitude.values:
    for lon in ds.longitude.values:

        X_ts = X_train.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y_train.sel(latitude=lat, longitude=lon).values

        valid = (
            ~np.isnan(y_ts) &
            ~np.isnan(X_ts).any(axis=1)
        )

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        if len(y_ts) <= LAG_DAYS:
            continue

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).ravel()

        X_seq, y_seq = build_sequences(X_scaled, y_scaled, LAG_DAYS)

        if len(y_seq) == 0:
            continue

        X_all.append(X_seq)
        y_all.append(y_seq)

X_train_all = np.concatenate(X_all)
y_train_all = np.concatenate(y_all)

print(f"Training samples: {X_train_all.shape[0]}")

# ============================================================
# MODEL
# ============================================================

model = Sequential([
    LSTM(64, input_shape=(LAG_DAYS, X_train_all.shape[2])),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")

model.fit(
    X_train_all,
    y_train_all,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

# ============================================================
# PREDICTION (NaN-SAFE)
# ============================================================

test_times = time.sel(time=test_idx)

preds = np.full(
    (len(ds.latitude), len(ds.longitude), len(test_times)),
    np.nan
)

for i, lat in enumerate(ds.latitude.values):
    for j, lon in enumerate(ds.longitude.values):

        X_ts = X.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y.sel(latitude=lat, longitude=lon).values

        valid = ~np.isnan(X_ts).any(axis=1)

        if valid.sum() <= LAG_DAYS:
            continue

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1))

        test_start = np.where(test_idx.values)[0][0]

        for k in range(len(test_times)):
            t = test_start + k
            if t - LAG_DAYS < 0:
                continue

            X_input = X_scaled[t-LAG_DAYS:t]

            if np.isnan(X_input).any():
                continue

            pred_scaled = model.predict(
                X_input.reshape(1, LAG_DAYS, -1),
                verbose=0
            )

            preds[i, j, k] = scaler_y.inverse_transform(pred_scaled)[0, 0]

# ============================================================
# OUTPUT
# ============================================================

pred_da = xr.DataArray(
    preds,
    dims=("latitude", "longitude", "time"),
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times
    },
    name=f"{POLLUTANT_NAME}_predicted"
)

# ============================================================
# EVALUATION (NaN-SAFE)
# ============================================================

error = pred_da - y_test

mae = float(abs(error).where(np.isfinite(error)).mean())
rmse = float(np.sqrt((error ** 2)).where(np.isfinite(error)).mean())
bias = float(error.where(np.isfinite(error)).mean())
corr = float(
    xr.corr(pred_da, y_test, dim="time")
    .where(np.isfinite(pred_da))
    .mean()
)

print("\nEvaluation (2017):")
print(f"  MAE:  {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  Bias: {bias:.4f}")
print(f"  Corr: {corr:.4f}")

# ============================================================
# SAVE RESULTS FOR MAPPING (NO PLOTTING)
# ============================================================

ds_out = xr.Dataset(
    {
        f"{POLLUTANT_NAME}_predicted": pred_da,
        f"{POLLUTANT_NAME}_observed": y_test,
        f"{POLLUTANT_NAME}_difference": pred_da - y_test,
    },
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times,
    },
)

# Add metadata (recommended)
ds_out[f"{POLLUTANT_NAME}_predicted"].attrs["description"] = "LSTM predicted pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_observed"].attrs["description"] = "Observed pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_difference"].attrs["description"] = "Prediction minus observation"

ds_out.attrs["model"] = "LSTM"
ds_out.attrs["lags_days"] = LAG_DAYS
ds_out.attrs["test_year"] = TEST_YEAR
ds_out.attrs["pollutant"] = POLLUTANT_NAME

# Save to NetCDF
output_file = f"{POLLUTANT_NAME}_LSTM_predictions_{TEST_YEAR}.nc"
ds_out.to_netcdf(output_file)

print(f"\nSaved results to: {output_file}")



Processing: NO
Training samples: 1139954
Epoch 1/40


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35624/35624 ━━━━━━━━━━━━━━━━━━━━ 133s 4ms/step - loss: 0.0027
Epoch 2/40
   29/35624 ━━━━━━━━━━━━━━━━━━━━ 2:11 4ms/step - loss: 0.0029 

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


35624/35624 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0026
Epoch 3/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0025
Epoch 4/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 133s 4ms/step - loss: 0.0025
Epoch 5/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 162s 5ms/step - loss: 0.0025
Epoch 6/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 192s 5ms/step - loss: 0.0025
Epoch 7/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/step - loss: 0.0024
Epoch 8/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/step - loss: 0.0024
Epoch 9/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/step - loss: 0.0024
Epoch 10/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 133s 4ms/step - loss: 0.0024
Epoch 11/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 133s 4ms/step - loss: 0.0024
Epoch 12/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 135s 4ms/step - loss: 0.0023
Epoch 13/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/step - loss: 0.0023
Epoch 14/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 133s 4ms/step - loss: 0.0023
Epoch 15/40
35624/35624 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/ste

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [3]:
# ============================================================
# DAILY POLLUTANT PREDICTION WITH LSTM (SINGLE POLLUTANT)
# NaNs are strictly ignored (no interpolation, no filling)
# ============================================================

import numpy as np
import xarray as xr

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# FILE PATH (CHANGE THIS PER RUN)
# ============================================================

POLLUTANT_NAME = "NO2"   # for naming outputs only
INPUT_FILE = r"D:\IPMA\Results\no2_fire_meteo_Iberia.nc"

# ============================================================
# USER SETTINGS
# ============================================================

POLLUTANT_VAR = "Mean"
TEST_YEAR = 2017
TEST_MONTH = None        # None or 1–12
LAG_DAYS = 14
EPOCHS = 40
BATCH_SIZE = 32

INPUT_VARS = [
    "Mean",
    "temp_Max",
    "wind_Max",
    "precip_Total_Precipitation",
    "frp_sum_Iberia"
]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def build_sequences(X, y, lags):
    X_out, y_out = [], []
    for i in range(lags, len(X)):
        X_out.append(X[i-lags:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)


def train_mask(time):
    return time.dt.year != TEST_YEAR


def test_mask(time):
    mask = time.dt.year == TEST_YEAR
    if TEST_MONTH is not None:
        mask = mask & (time.dt.month == TEST_MONTH)
    return mask

# ============================================================
# LOAD DATA
# ============================================================

print(f"\n==============================")
print(f"Processing: {POLLUTANT_NAME}")
print(f"==============================")

ds = xr.open_dataset(INPUT_FILE)

# -------------------------------
# CHECK VARIABLES
# -------------------------------

for var in INPUT_VARS:
    if var not in ds:
        raise KeyError(f"{var} not found in {INPUT_FILE}")

y = ds[POLLUTANT_VAR]
X = xr.merge([ds[var] for var in INPUT_VARS])

time = ds.time
train_idx = train_mask(time)
test_idx = test_mask(time)

X_train = X.sel(time=train_idx)
y_train = y.sel(time=train_idx)
X_test = X.sel(time=test_idx)
y_test = y.sel(time=test_idx)

# ============================================================
# BUILD TRAINING DATA (NaN-SAFE)
# ============================================================

X_all, y_all = [], []

for lat in ds.latitude.values:
    for lon in ds.longitude.values:

        X_ts = X_train.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y_train.sel(latitude=lat, longitude=lon).values

        valid = (
            ~np.isnan(y_ts) &
            ~np.isnan(X_ts).any(axis=1)
        )

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        if len(y_ts) <= LAG_DAYS:
            continue

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).ravel()

        X_seq, y_seq = build_sequences(X_scaled, y_scaled, LAG_DAYS)

        if len(y_seq) == 0:
            continue

        X_all.append(X_seq)
        y_all.append(y_seq)

X_train_all = np.concatenate(X_all)
y_train_all = np.concatenate(y_all)

print(f"Training samples: {X_train_all.shape[0]}")

# ============================================================
# MODEL
# ============================================================

model = Sequential([
    LSTM(64, input_shape=(LAG_DAYS, X_train_all.shape[2])),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")

model.fit(
    X_train_all,
    y_train_all,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

# ============================================================
# PREDICTION (NaN-SAFE)
# ============================================================

test_times = time.sel(time=test_idx)

preds = np.full(
    (len(ds.latitude), len(ds.longitude), len(test_times)),
    np.nan
)

for i, lat in enumerate(ds.latitude.values):
    for j, lon in enumerate(ds.longitude.values):

        X_ts = X.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y.sel(latitude=lat, longitude=lon).values

        valid = ~np.isnan(X_ts).any(axis=1)

        if valid.sum() <= LAG_DAYS:
            continue

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1))

        test_start = np.where(test_idx.values)[0][0]

        for k in range(len(test_times)):
            t = test_start + k
            if t - LAG_DAYS < 0:
                continue

            X_input = X_scaled[t-LAG_DAYS:t]

            if np.isnan(X_input).any():
                continue

            pred_scaled = model.predict(
                X_input.reshape(1, LAG_DAYS, -1),
                verbose=0
            )

            preds[i, j, k] = scaler_y.inverse_transform(pred_scaled)[0, 0]

# ============================================================
# OUTPUT
# ============================================================

pred_da = xr.DataArray(
    preds,
    dims=("latitude", "longitude", "time"),
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times
    },
    name=f"{POLLUTANT_NAME}_predicted"
)

# ============================================================
# EVALUATION (NaN-SAFE)
# ============================================================

error = pred_da - y_test

mae = float(abs(error).where(np.isfinite(error)).mean())
rmse = float(np.sqrt((error ** 2)).where(np.isfinite(error)).mean())
bias = float(error.where(np.isfinite(error)).mean())
corr = float(
    xr.corr(pred_da, y_test, dim="time")
    .where(np.isfinite(pred_da))
    .mean()
)

print("\nEvaluation (2017):")
print(f"  MAE:  {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  Bias: {bias:.4f}")
print(f"  Corr: {corr:.4f}")

# ============================================================
# SAVE RESULTS FOR MAPPING (NO PLOTTING)
# ============================================================

ds_out = xr.Dataset(
    {
        f"{POLLUTANT_NAME}_predicted": pred_da,
        f"{POLLUTANT_NAME}_observed": y_test,
        f"{POLLUTANT_NAME}_difference": pred_da - y_test,
    },
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times,
    },
)

# Add metadata (recommended)
ds_out[f"{POLLUTANT_NAME}_predicted"].attrs["description"] = "LSTM predicted pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_observed"].attrs["description"] = "Observed pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_difference"].attrs["description"] = "Prediction minus observation"

ds_out.attrs["model"] = "LSTM"
ds_out.attrs["lags_days"] = LAG_DAYS
ds_out.attrs["test_year"] = TEST_YEAR
ds_out.attrs["pollutant"] = POLLUTANT_NAME

# Save to NetCDF
output_file = f"{POLLUTANT_NAME}_LSTM_predictions_{TEST_YEAR}.nc"
ds_out.to_netcdf(output_file)

print(f"\nSaved results to: {output_file}")




Processing: NO2
Training samples: 1139999


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0055
Epoch 2/40
   28/35625 ━━━━━━━━━━━━━━━━━━━━ 2:11 4ms/step - loss: 0.0054 

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0053
Epoch 3/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0052
Epoch 4/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/step - loss: 0.0052
Epoch 5/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0051
Epoch 6/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 143s 4ms/step - loss: 0.0051
Epoch 7/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 200s 6ms/step - loss: 0.0050
Epoch 8/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 197s 6ms/step - loss: 0.0050
Epoch 9/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 142s 4ms/step - loss: 0.0049
Epoch 10/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0049
Epoch 11/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0048
Epoch 12/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0048
Epoch 13/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0047
Epoch 14/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 134s 4ms/step - loss: 0.0047
Epoch 15/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 158s 4ms/ste

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
# ============================================================
# DAILY POLLUTANT PREDICTION WITH LSTM (SINGLE POLLUTANT)
# NaNs are strictly ignored (no interpolation, no filling)
# ============================================================

import numpy as np
import xarray as xr

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# FILE PATH (CHANGE THIS PER RUN)
# ============================================================

POLLUTANT_NAME = "PM2.5"   # for naming outputs only
INPUT_FILE = r"D:\IPMA\Results\pm2p5_fire_meteo_Iberia.nc"

# ============================================================
# USER SETTINGS
# ============================================================

POLLUTANT_VAR = "Mean"
TEST_YEAR = 2017
TEST_MONTH = None        # None or 1–12
LAG_DAYS = 14
EPOCHS = 40
BATCH_SIZE = 32

INPUT_VARS = [
    "Mean",
    "temp_Max",
    "wind_Max",
    "precip_Total_Precipitation",
    "frp_sum_Iberia"
]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def build_sequences(X, y, lags):
    X_out, y_out = [], []
    for i in range(lags, len(X)):
        X_out.append(X[i-lags:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)


def train_mask(time):
    return time.dt.year != TEST_YEAR


def test_mask(time):
    mask = time.dt.year == TEST_YEAR
    if TEST_MONTH is not None:
        mask = mask & (time.dt.month == TEST_MONTH)
    return mask

# ============================================================
# LOAD DATA
# ============================================================

print(f"\n==============================")
print(f"Processing: {POLLUTANT_NAME}")
print(f"==============================")

ds = xr.open_dataset(INPUT_FILE)

# -------------------------------
# CHECK VARIABLES
# -------------------------------

for var in INPUT_VARS:
    if var not in ds:
        raise KeyError(f"{var} not found in {INPUT_FILE}")

y = ds[POLLUTANT_VAR]
X = xr.merge([ds[var] for var in INPUT_VARS])

time = ds.time
train_idx = train_mask(time)
test_idx = test_mask(time)

X_train = X.sel(time=train_idx)
y_train = y.sel(time=train_idx)
X_test = X.sel(time=test_idx)
y_test = y.sel(time=test_idx)

# ============================================================
# BUILD TRAINING DATA (NaN-SAFE)
# ============================================================

X_all, y_all = [], []

for lat in ds.latitude.values:
    for lon in ds.longitude.values:

        X_ts = X_train.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y_train.sel(latitude=lat, longitude=lon).values

        valid = (
            ~np.isnan(y_ts) &
            ~np.isnan(X_ts).any(axis=1)
        )

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        if len(y_ts) <= LAG_DAYS:
            continue

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).ravel()

        X_seq, y_seq = build_sequences(X_scaled, y_scaled, LAG_DAYS)

        if len(y_seq) == 0:
            continue

        X_all.append(X_seq)
        y_all.append(y_seq)

X_train_all = np.concatenate(X_all)
y_train_all = np.concatenate(y_all)

print(f"Training samples: {X_train_all.shape[0]}")

# ============================================================
# MODEL
# ============================================================

model = Sequential([
    LSTM(64, input_shape=(LAG_DAYS, X_train_all.shape[2])),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")

model.fit(
    X_train_all,
    y_train_all,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

# ============================================================
# PREDICTION (NaN-SAFE)
# ============================================================

test_times = time.sel(time=test_idx)

preds = np.full(
    (len(ds.latitude), len(ds.longitude), len(test_times)),
    np.nan
)

for i, lat in enumerate(ds.latitude.values):
    for j, lon in enumerate(ds.longitude.values):

        X_ts = X.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y.sel(latitude=lat, longitude=lon).values

        valid = ~np.isnan(X_ts).any(axis=1)

        if valid.sum() <= LAG_DAYS:
            continue

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1))

        test_start = np.where(test_idx.values)[0][0]

        for k in range(len(test_times)):
            t = test_start + k
            if t - LAG_DAYS < 0:
                continue

            X_input = X_scaled[t-LAG_DAYS:t]

            if np.isnan(X_input).any():
                continue

            pred_scaled = model.predict(
                X_input.reshape(1, LAG_DAYS, -1),
                verbose=0
            )

            preds[i, j, k] = scaler_y.inverse_transform(pred_scaled)[0, 0]

# ============================================================
# OUTPUT
# ============================================================

pred_da = xr.DataArray(
    preds,
    dims=("latitude", "longitude", "time"),
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times
    },
    name=f"{POLLUTANT_NAME}_predicted"
)

# ============================================================
# EVALUATION (NaN-SAFE)
# ============================================================

error = pred_da - y_test

mae = float(abs(error).where(np.isfinite(error)).mean())
rmse = float(np.sqrt((error ** 2)).where(np.isfinite(error)).mean())
bias = float(error.where(np.isfinite(error)).mean())
corr = float(
    xr.corr(pred_da, y_test, dim="time")
    .where(np.isfinite(pred_da))
    .mean()
)

print("\nEvaluation (2017):")
print(f"  MAE:  {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  Bias: {bias:.4f}")
print(f"  Corr: {corr:.4f}")

# ============================================================
# SAVE RESULTS FOR MAPPING (NO PLOTTING)
# ============================================================

ds_out = xr.Dataset(
    {
        f"{POLLUTANT_NAME}_predicted": pred_da,
        f"{POLLUTANT_NAME}_observed": y_test,
        f"{POLLUTANT_NAME}_difference": pred_da - y_test,
    },
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times,
    },
)

# Add metadata (recommended)
ds_out[f"{POLLUTANT_NAME}_predicted"].attrs["description"] = "LSTM predicted pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_observed"].attrs["description"] = "Observed pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_difference"].attrs["description"] = "Prediction minus observation"

ds_out.attrs["model"] = "LSTM"
ds_out.attrs["lags_days"] = LAG_DAYS
ds_out.attrs["test_year"] = TEST_YEAR
ds_out.attrs["pollutant"] = POLLUTANT_NAME

# Save to NetCDF
output_file = f"{POLLUTANT_NAME}_LSTM_predictions_{TEST_YEAR}.nc"
ds_out.to_netcdf(output_file)

print(f"\nSaved results to: {output_file}")



C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):



Processing: PM2.5
Training samples: 1139997


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 161s 4ms/step - loss: 0.0013
Epoch 2/40
   11/35625 ━━━━━━━━━━━━━━━━━━━━ 3:04 5ms/step - loss: 0.0037  

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


35625/35625 ━━━━━━━━━━━━━━━━━━━━ 168s 5ms/step - loss: 0.0013
Epoch 3/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 207s 6ms/step - loss: 0.0012
Epoch 4/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 217s 6ms/step - loss: 0.0012
Epoch 5/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 216s 6ms/step - loss: 0.0012
Epoch 6/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 163s 5ms/step - loss: 0.0012
Epoch 7/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 175s 5ms/step - loss: 0.0012
Epoch 8/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 189s 5ms/step - loss: 0.0012
Epoch 9/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 167s 5ms/step - loss: 0.0012
Epoch 10/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 200s 5ms/step - loss: 0.0012
Epoch 11/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 177s 5ms/step - loss: 0.0012
Epoch 12/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 166s 5ms/step - loss: 0.0011
Epoch 13/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 200s 5ms/step - loss: 0.0011
Epoch 14/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 207s 5ms/step - loss: 0.0011
Epoch 15/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 168s 5ms/ste

In [ ]:
# ============================================================
# DAILY POLLUTANT PREDICTION WITH LSTM (SINGLE POLLUTANT)
# NaNs are strictly ignored (no interpolation, no filling)
# ============================================================

import numpy as np
import xarray as xr

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# FILE PATH (CHANGE THIS PER RUN)
# ============================================================

POLLUTANT_NAME = "PM10"   # for naming outputs only
INPUT_FILE = r"D:\IPMA\Results\pm10_fire_meteo_Iberia.nc"

# ============================================================
# USER SETTINGS
# ============================================================

POLLUTANT_VAR = "Mean"
TEST_YEAR = 2017
TEST_MONTH = None        # None or 1–12
LAG_DAYS = 14
EPOCHS = 40
BATCH_SIZE = 32

INPUT_VARS = [
    "Mean",
    "temp_Max",
    "wind_Max",
    "precip_Total_Precipitation",
    "frp_sum_Iberia"
]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def build_sequences(X, y, lags):
    X_out, y_out = [], []
    for i in range(lags, len(X)):
        X_out.append(X[i-lags:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)


def train_mask(time):
    return time.dt.year != TEST_YEAR


def test_mask(time):
    mask = time.dt.year == TEST_YEAR
    if TEST_MONTH is not None:
        mask = mask & (time.dt.month == TEST_MONTH)
    return mask

# ============================================================
# LOAD DATA
# ============================================================

print(f"\n==============================")
print(f"Processing: {POLLUTANT_NAME}")
print(f"==============================")

ds = xr.open_dataset(INPUT_FILE)

# -------------------------------
# CHECK VARIABLES
# -------------------------------

for var in INPUT_VARS:
    if var not in ds:
        raise KeyError(f"{var} not found in {INPUT_FILE}")

y = ds[POLLUTANT_VAR]
X = xr.merge([ds[var] for var in INPUT_VARS])

time = ds.time
train_idx = train_mask(time)
test_idx = test_mask(time)

X_train = X.sel(time=train_idx)
y_train = y.sel(time=train_idx)
X_test = X.sel(time=test_idx)
y_test = y.sel(time=test_idx)

# ============================================================
# BUILD TRAINING DATA (NaN-SAFE)
# ============================================================

X_all, y_all = [], []

for lat in ds.latitude.values:
    for lon in ds.longitude.values:

        X_ts = X_train.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y_train.sel(latitude=lat, longitude=lon).values

        valid = (
            ~np.isnan(y_ts) &
            ~np.isnan(X_ts).any(axis=1)
        )

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        if len(y_ts) <= LAG_DAYS:
            continue

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).ravel()

        X_seq, y_seq = build_sequences(X_scaled, y_scaled, LAG_DAYS)

        if len(y_seq) == 0:
            continue

        X_all.append(X_seq)
        y_all.append(y_seq)

X_train_all = np.concatenate(X_all)
y_train_all = np.concatenate(y_all)

print(f"Training samples: {X_train_all.shape[0]}")

# ============================================================
# MODEL
# ============================================================

model = Sequential([
    LSTM(64, input_shape=(LAG_DAYS, X_train_all.shape[2])),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")

model.fit(
    X_train_all,
    y_train_all,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

# ============================================================
# PREDICTION (NaN-SAFE)
# ============================================================

test_times = time.sel(time=test_idx)

preds = np.full(
    (len(ds.latitude), len(ds.longitude), len(test_times)),
    np.nan
)

for i, lat in enumerate(ds.latitude.values):
    for j, lon in enumerate(ds.longitude.values):

        X_ts = X.sel(latitude=lat, longitude=lon).to_array().values.T
        y_ts = y.sel(latitude=lat, longitude=lon).values

        valid = ~np.isnan(X_ts).any(axis=1)

        if valid.sum() <= LAG_DAYS:
            continue

        X_ts = X_ts[valid]
        y_ts = y_ts[valid]

        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_scaled = scaler_X.fit_transform(X_ts)
        y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1))

        test_start = np.where(test_idx.values)[0][0]

        for k in range(len(test_times)):
            t = test_start + k
            if t - LAG_DAYS < 0:
                continue

            X_input = X_scaled[t-LAG_DAYS:t]

            if np.isnan(X_input).any():
                continue

            pred_scaled = model.predict(
                X_input.reshape(1, LAG_DAYS, -1),
                verbose=0
            )

            preds[i, j, k] = scaler_y.inverse_transform(pred_scaled)[0, 0]

# ============================================================
# OUTPUT
# ============================================================

pred_da = xr.DataArray(
    preds,
    dims=("latitude", "longitude", "time"),
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times
    },
    name=f"{POLLUTANT_NAME}_predicted"
)

# ============================================================
# EVALUATION (NaN-SAFE)
# ============================================================

error = pred_da - y_test

mae = float(abs(error).where(np.isfinite(error)).mean())
rmse = float(np.sqrt((error ** 2)).where(np.isfinite(error)).mean())
bias = float(error.where(np.isfinite(error)).mean())
corr = float(
    xr.corr(pred_da, y_test, dim="time")
    .where(np.isfinite(pred_da))
    .mean()
)

print("\nEvaluation (2017):")
print(f"  MAE:  {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  Bias: {bias:.4f}")
print(f"  Corr: {corr:.4f}")

# ============================================================
# SAVE RESULTS FOR MAPPING (NO PLOTTING)
# ============================================================

ds_out = xr.Dataset(
    {
        f"{POLLUTANT_NAME}_predicted": pred_da,
        f"{POLLUTANT_NAME}_observed": y_test,
        f"{POLLUTANT_NAME}_difference": pred_da - y_test,
    },
    coords={
        "latitude": ds.latitude,
        "longitude": ds.longitude,
        "time": test_times,
    },
)

# Add metadata (recommended)
ds_out[f"{POLLUTANT_NAME}_predicted"].attrs["description"] = "LSTM predicted pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_observed"].attrs["description"] = "Observed pollutant concentration"
ds_out[f"{POLLUTANT_NAME}_difference"].attrs["description"] = "Prediction minus observation"

ds_out.attrs["model"] = "LSTM"
ds_out.attrs["lags_days"] = LAG_DAYS
ds_out.attrs["test_year"] = TEST_YEAR
ds_out.attrs["pollutant"] = POLLUTANT_NAME

# Save to NetCDF
output_file = f"{POLLUTANT_NAME}_LSTM_predictions_{TEST_YEAR}.nc"
ds_out.to_netcdf(output_file)

print(f"\nSaved results to: {output_file}")



Processing: PM10
Training samples: 1139998


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0014
Epoch 2/40
   31/35625 ━━━━━━━━━━━━━━━━━━━━ 2:03 3ms/step - loss: 0.0016     

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0013
Epoch 3/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0013
Epoch 4/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0013
Epoch 5/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0013
Epoch 6/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0013
Epoch 7/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0012
Epoch 8/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0012
Epoch 9/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0012
Epoch 10/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0012
Epoch 11/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0012
Epoch 12/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0012
Epoch 13/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0012
Epoch 14/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 141s 4ms/step - loss: 0.0012
Epoch 15/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/ste

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
